In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import cv2 as cv
import glob
from google.colab.patches import cv_imshow

#중단기준설정(오차의 변화율, 최대반복횟수)
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
  #cv.TERM_CRITERIA_EPS: 오차변화율(epsilon) 기준
  #cv.TERM_CRITERIA_MAX_ITER: 최대 반복 횟수

#object point 설정
objp = np.zeros((6*8,3), np.float32)
  #42x3크기의 영행렬 생성
objp[:,:2] = np.mgrid[0:8,0:6].T.reshape(-1,2)
  #np.mgrid: 좌표격자 생성
  #T.reshape(-1,2): (*행 2열)로 재구성하라

objpoints = []
  #3D공간좌표(실제세계에서의 점)
imgpoints = []
  #2D픽셀좌표(이미지평면상에서의 점)

images = glob.glob('/content/gdrive/MyDrive/MachineVision/assignment1/*.jpeg')
  #해당위치에서 이름이 left로 시작하고 확장자가 jpg인 모든 파일을 찾아 리스트로 만들기 (정렬은 보장못함)

images

In [ ]:
for fname in images:
  #이미지 불러오기
  img = cv.imread(fname)
  gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    #코너검출에서는 밝기변화만 중요하고 색변화는 방해가 될 수 있기에 그레이스케일로 변환

  #체스보드 코너찾기
  ret, corners = cv.findChessboardCorners(gray, (8,6), None)
    #ret: 성공여부반환
    #corners:

  if ret == True:
    objpoints.append(objp)
    corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
    imgpoints.append(corners2)
    cv.drawChessboardCorners(img, (8,6), corners2, ret)
    cv_imshow(img)



In [12]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [6]:
mtx

array([[4.35209167e+03, 0.00000000e+00, 2.15976062e+03],
       [0.00000000e+00, 4.35233334e+03, 2.84897573e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [7]:
dist

array([[ 3.23621588e-01, -2.39630637e+00, -3.44703186e-03,
         3.44988507e-04,  4.60405085e+00]])

In [ ]:
rvecs

In [ ]:
tvecs

In [10]:
def draw(img, imgpts):
  imgpts = np.int32(imgpts).reshape(-1,2)
  #아랫면 그리기
  img = cv.drawContours(img, [imgpts[:3]], -1, (0,255,0), -3)
  #옆면 그리기
  for i,j in zip(range(3), [3,3,3]):
    img = cv.line(img, tuple(imgpts[i]), tuple(imgpts[j]), (255,0,0), 3)
  return img

In [ ]:
startPoint = np.array([2,2,0], dtype=np.float32)
x0, y0, z0 = startPoint
dx, dy, dz = 1, 2, 3

axis = np.float32([
    [x0,     y0,     z0    ],
    [x0,     y0+dy,  z0    ],
    [x0+dx,  y0+dy,  z0    ],
    [x0,     y0,     z0-dz ],
])

i = 0
for fname in glob.glob('/content/gdrive/MyDrive/MachineVision/assignment1/*.jpeg'):
  img = cv.imread(fname)
  gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
  ret, corners = cv.findChessboardCorners(gray, (8,6), None)
  if ret == True:
    imgpts, jac = cv.projectPoints(axis, rvecs[i], tvecs[i], mtx, dist)
    i = i+1
    img = draw(img, imgpts)
    cv_imshow(img)